In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

C:\Users\TagB\.conda\envs\PyTorch\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
metadata_path = r"R:\Speech and Craniofacial analysis\Collected Data\UMich Data\April_16_2025\Jacqueline\April16_allSegments_processed.xlsx"
metadata = pd.read_excel(metadata_path)
metadata = metadata[~metadata['vowel'].str.contains('sentence', case=False, na=False)]  # Filter out sentences
metadata = metadata.reset_index(drop=True)

In [4]:
import scipy
import warnings
import librosa
from scipy import signal
import pandas as pd
import numpy as np

def logEnt(sigHist):
  s = np.log(sigHist+1E-6)**2
  return s.sum()

def ShanonEnt(y):
  binwidth = (3.49*y.std())/(np.power(len(y),1/3))
  bins=np.arange(y.min(), y.max() + binwidth, binwidth)
  c= np.histogram(y,bins=bins)
  pdf = c[0]/c[0].sum()
  Ent = scipy.stats.entropy(pdf)
  return [pdf,Ent]

def bandpower(x, fs, fmin, fmax):
    f, Pxx = scipy.signal.periodogram(x, fs=fs)
    Pxx = Pxx/Pxx.mean()
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    return np.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

def featureExtractor(inputSignal,sr):
    d = {}

    d["shEnt"] = []
    d["logEnt"] = []
  
    d["bandPow0_10"] = []
    d["bandPow10_20"] = []
    d["bandPow20_30"] = []
    d["bandPow30_40"] = []
    d["bandPow40_50"] = []
    d["bandPow50_60"] = []
  
    for z in range(13): d["mfcc_"+str(z)] = []
    for z in range(13): d["dmfcc_"+str(z)] = []
    for z in range(13): d["ddmfcc_"+str(z)] = []
  
    for z in range(12): d["chroma"+str(z)] = []
  
    d["spec_centroid"] = []
    d["spec_bandwith"] = []
    d["spec_flatness"] = []
    d["spec_rolloff"] = []
    for z in range(7): d["spec_contrast"+str(z)] = []
  
    for z in range(4): d["poly"+str(z)] = []
  
    d["zero_crossing"] =[]
  
    d["tempgram_mean"] = []
    d["tempgram_var"] = []
    d["tempgram_kurt"] = []
    d["tempgram_skew"] = []
  
    d["mean"] = []
    d["var"] = []
    d["skew"] = []
    d["kurt"] = []
    d["range"] = []
    d["diff_SMA"] = []
  
    d['PXX_skew'] = []
    d['PXX_RMS'] = []
    d["PXX_mean"] = []
  
    d['PXX_shEnt'] = []
    d['PXX_logEnt'] = []
    d["PXX_range"] = []
    d["PXX_diff_SMA"] = []
  
    s = inputSignal
    pdf,Ent = ShanonEnt(s) #Shannon Entropy   

    # Statistical features from signal
    d["mean"].append(np.mean(s))
    d["var"].append(np.var(s))
    d["skew"].append(scipy.stats.skew(s))
    d["kurt"].append(scipy.stats.kurtosis(s))
    d["range"].append(s.max()-s.min())
    d["diff_SMA"].append(abs(np.diff(s,1)).sum())

    d["shEnt"].append(Ent) #Shanon Entropy
    d["logEnt"].append(logEnt(pdf)) #Log Entropy

    s = (s-s.mean())/(s.std())

    # MFCC/DeltaMFCC/ DeltaDeltaMFCC
    mfccs = librosa.feature.mfcc(y=s, sr=sr, n_mfcc=13)
    m = np.median(mfccs,axis=1)

    mfcc_delta = np.diff(mfccs,1)
    mdelta = np.median(mfcc_delta,axis=1)

    mfcc_deltadelta = np.diff(mfcc_delta,1)
    mdeltadelta = np.median(mfcc_deltadelta,axis=1)

    for z in range(13): d["mfcc_"+str(z)].append(m[z])
    for z in range(13): d["dmfcc_"+str(z)].append(mdelta[z])
    for z in range(13): d["ddmfcc_"+str(z)].append(mdeltadelta[z])

    # Chroma-related features
    S = np.abs(librosa.stft(s))**2
    chroma = librosa.feature.chroma_stft(S=S, sr=sr)
    chMax = np.median(chroma,axis=1)

    for z in range(12): d["chroma"+str(z)].append(chMax[z])

    # Spectral-related features
    contr = librosa.feature.spectral_contrast(y=s,sr=sr,n_bands=6)
    contrMed = np.median(contr,axis=1)
    for z in range(7): d["spec_contrast"+str(z)].append(contrMed[z])

    d["spec_centroid"].append(np.median(librosa.feature.spectral_centroid(s,sr=sr)))
    d["spec_bandwith"].append(np.median(librosa.feature.spectral_bandwidth(s,sr=sr)))
    d["spec_flatness"].append(np.median(librosa.feature.spectral_flatness(s)))
    d["spec_rolloff"].append(np.median(librosa.feature.spectral_rolloff(y=s, sr=sr)))

    # Get coefficients of fitting an 3rd-order polynomial to the columns of a spectrogram.
    polyFeat = librosa.feature.poly_features(y=s,sr=sr,order=3)
    polyMed = np.median(polyFeat,axis=1)
    for z in range(4): d["poly"+str(z)].append(polyMed[z])

    # Zero crossing rate
    d["zero_crossing"].append(np.median(librosa.feature.zero_crossing_rate(y=s,frame_length= len(s)),axis=1)[0])

    # Statistical features from tempogram
    tempGram = librosa.feature.tempogram(y=s, sr=sr)
    d["tempgram_mean"].append(np.mean(tempGram))
    d["tempgram_var"].append(np.var(tempGram))
    d["tempgram_kurt"].append(scipy.stats.kurtosis(tempGram).mean())
    d["tempgram_skew"].append(scipy.stats.skew(tempGram).mean())

    # Band powers
    d["bandPow0_10"].append(bandpower(s, sr, 0.1, int(sr/12)))
    d["bandPow10_20"].append(bandpower(s, sr, int(sr/12), int(2*sr/12)))
    d["bandPow20_30"].append(bandpower(s, sr, int(2*sr/12), int(3*sr/12)))
    d["bandPow30_40"].append(bandpower(s, sr, int(3*sr/12), int(4*sr/12)))
    d["bandPow40_50"].append(bandpower(s, sr, int(4*sr/12), int(5*sr/12)))
    d["bandPow50_60"].append(bandpower(s, sr, int(5*sr/12), int(sr/2)))


    f,pxx = signal.welch((s-s.mean())/s.std(), fs=sr, window='hann', nperseg=16,noverlap=15, nfft=2048)
    fIntent = np.where(f>7200)
    pxx = pxx[0:fIntent[0][0]-1]

    d['PXX_skew'].append(scipy.stats.skew(pxx))
    d['PXX_RMS'].append(np.sqrt(np.mean(pxx**2)))
    d['PXX_mean'].append(pxx.mean())
    d["PXX_range"].append(pxx.max()-pxx.min())

    pdf,Ent = ShanonEnt(pxx) #Shannon Entropy
    d["PXX_shEnt"].append(Ent) #Shanon Entropy
    d["PXX_logEnt"].append(logEnt(pdf)) #Log Entropy
    d["PXX_diff_SMA"].append(abs(np.diff(pxx,1)).sum())
    return d

In [ ]:
import torchaudio
warnings.filterwarnings('ignore')

df_features_all = pd.DataFrame()
k = 0

for idx, row in tqdm(metadata.iterrows()):
    file_path = row['location']
    waveform, sample_rate = torchaudio.load(file_path)
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0)
    waveform = waveform.squeeze().numpy()
    df_feature = featureExtractor(np.array(waveform),sample_rate)
    df_dummy = pd.concat([pd.DataFrame(metadata.loc[k]).transpose().reset_index(),pd.DataFrame(df_feature).reset_index(drop=True)],axis=1)
    df_features_all = pd.concat([df_features_all,df_dummy],ignore_index = True)
    
    k+= 1

175it [03:28,  1.19s/it]


In [6]:
df_features_all.to_excel('DemographPlusAcousticFeatures_UMich_SRK.xlsx')